# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [ ]:
import sqlite3

con = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/lab 6/Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT firstname, lastname
FROM Customer 
WHERE country=?
''', ('Canada', ))

print(cur.fetchmany(5))

[('François', 'Tremblay'), ('Mark', 'Philips'), ('Jennifer', 'Peterson'), ('Robert', 'Brown'), ('Edward', 'Francis')]


2. Найти и вывести на экран названия всех альбомов группы Accept

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [2]:
import sqlite3

con = sqlite3.connect('recipes.db')
cur = con.cursor()



2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [14]:
def droppp (cur):
  #pass
  cur.execute("""DROP TABLE IF EXISTS Recipe;""")

def create_table(cur, create_table_sql):
  cur.execute(create_table_sql)

message="""
    CREATE TABLE Recipe(
        id INTEGER PRIMARY KEY, 
        name VARCHAR(50) NOT NULL, 
        minutes INTEGER,
        submitted DATE NOT NULL,
        desciption VARCHAR(200),
        n_ingredients INTEGER
                                    ); """

droppp(cur)
create_table (cur, message)

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [15]:
def droppp (cur):
  #pass
  cur.execute("""DROP TABLE IF EXISTS Review;""")

def create_table(cur, create_table_sql):
  cur.execute(create_table_sql)

message="""
    CREATE TABLE Review(
        id INTEGER PRIMARY KEY, 
        user_id INTEGER NOT NULL,
        recipe_id INTEGER NOT NULL,
        date DATE NOT NULL,
        rating INTEGER NOT NULL,
        review VARCHAR(250),
        FOREIGN KEY (recipe_id) REFERENCES Recipe(id)

                                    ); """

droppp(cur)
create_table (cur, message)

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [19]:
import pandas as pd
import csv
from sqlalchemy.types import Integer

# reviews = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/lab 3/reviews_sample.csv' , names=['id', 'user_id','recipe_id', 'date','rating', 'review'  ] , sep=',')
# recipes = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/recipes_sample_with_filled_nsteps.csv', sep=',' , usecols=['id', 'name', 'minutes', 'submitted','description', 'n_ingredients'] )


# #cur.executemany("INSERT INTO Review VALUES (?, ?, ? , ? , ?, ?)", reviews.values)


# #cur.executemany("INSERT INTO Recipe VALUES (?, ?, ? , ? , ?, ?)", recipes.values)



# con.commit()


# #cur.execute("SELECT * FROM Review")


reviews_sample = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lab 3/reviews_sample.csv', sep=',')
reviews_sample = reviews_sample.set_index('Unnamed: 0')
#reviews_sample


recipes_sample_with_tags_ingredients = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lab 4/recipes_sample_with_filled_nsteps.csv', sep=',')
#recipes_sample_with_tags_ingredients

# for index, row in reviews_sample.iterrows():
#     cur.execute("""
#         INSERT INTO Review(id, user_id, recipe_id, date, rating, review) 
#         VALUES (?, ?, ?, ?, ?, ?)
#     """, (index, row[0], row[1], row[2], row[3], row[4]))
# con.commit()


for index, row in recipes_sample_with_tags_ingredients.iterrows():
    print (row)
    cur.execute("""
        INSERT INTO Recipe(id, name, minutes, submitted, desciption, n_ingredients) 
        VALUES (?, ?, ?, ?, ?, ?)
    """, (row[1], row[0], row[2], row[4], row[6], row[7]))
con.commit()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 4))



44123                                                                                                                                                                                                                                                                                                                                                                                     67664
george s at the cove  black bean soup                                                                                                                                                                                                                                                                                                                        healthy for them  yogurt popsicles
90                                                                                                                                                                                                                                      

IntegrityError: ignored

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [17]:

cur.execute('''
    SELECT * 
    FROM Recipe 
    WHERE n_ingredients = ?
''', (10, ))
print(cur.fetchmany(5))

[]


6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [20]:
cur.execute('''
    SELECT name, MAX(minutes) 
    FROM Recipe 
''')
print(cur.fetchone())

(None, None)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [21]:
try:
    id_recipe = int(input("Введите индекс рецепта: "))
    
    cur.execute('''
        SELECT *
        FROM Recipe
        WHERE id = ?
    ''', (id_recipe, ))
    result = cur.fetchone()
    
    print(result) if result is not None else print("Рецепт не существует")
except ValueError:
    print("Неверный формат индекса рецепта")

KeyboardInterrupt: ignored

8. Найдите кол-во отзывов с рейтингом 5.

In [ ]:
cur.execute('''
    SELECT COUNT(*)
    FROM Review
    WHERE rating = ?
''', (5, ))
print(cur.fetchone()[0])

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [ ]:
res = cur.execute('''
    SELECT COUNT(*) OVER()
    FROM recipe rec
    LEFT JOIN review rev
    ON rec.id = rev.recipe_id
    GROUP BY rec.id
    HAVING 0 = COUNT(rev.rating < ?)
    LIMIT 1
''', (4, ))

print(res.fetchone()[0])

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [ ]:

result = cur.execute("""
    SELECT COUNT(*)
    FROM Recipe
    WHERE minutes >= ?
    AND submitted BETWEEN ? AND ?
""", (15, '2010-01-01', '2010-12-31'))
print(result.fetchone()[0])

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [ ]:
res = cur.execute('''
    SELECT rec.id, rec.name, rev.user_id, rev.date, rev.rating
    FROM recipe rec
    LEFT JOIN review rev
    ON rec.id = rev.recipe_id
    WHERE rec.n_ingredients >= ?
    ORDER BY rec.id
''', (3, ))

print(res.fetchmany(1000))

cur.close()
conn.close()